In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
#path
%cd /content/gdrive/My Drive/Research/UOH/AGenericALSSL_ESwA2023/

In [ ]:
!pip install tensorflow-gpu==2.6
!pip install keras==2.6 -q
!pip install scikit-learn==0.22.2.post1 -q
!pip install transformers==4.15.0 -q
!pip install xgboost==1.7.3 -q
!pip install numpy==1.21.2 -q

import sklearn
print('\nsklearn:', sklearn.__version__)
import torch 
print('torch:', torch.__version__)
import transformers
print('transformers:', transformers.__version__)

In [ ]:
#CONFIGURATION
import os
from utils import *
seed_everything()

GRID_PARAMS = True #True False
PGE = None #exp
RESULTS_OPT = True
PL_AL_OPT = True
SSL_OPT = True

DATASET = 'HGGB'
FILENAME = 'FUMADOR' #OBESIDAD OBESIDAD_TIPOS FUMADOR
#DATASET = 'UCI'
#FILENAME = 'AMAZON' #UCI YELP IMDB

BATCH = 64 #32 64 128 
SSL_TYPE = 'del-each-one' #'all' 'add-each-one' 'del-each-one' 'del-clustering'
THR_PRB = 0.9 #0.85 0.9 0.95

In [ ]:
DOMAIN = 'texts'
RUNS = 1
FOLDS = 5
METHOD = 'tfidf'
GPU = torch.cuda.is_available()
if GPU:
    print('GPU', torch.cuda.get_device_name(0))

with open( os.path.join( os.getcwd(), DOMAIN, 'data', 'snippets_procesados_'+FILENAME),  'rb') as a:
    data_aux = pickle.load(a)    
    data_aux = sorted(data_aux, key = lambda x:x[0], reverse = False)

DATA = np.array([snippet for snippet, classe in data_aux])[:200]
CLASSES = np.array([classe for snippet, classe in data_aux])[:200]
NCLASSES = len(set(CLASSES))
folds = KFold(n_splits = FOLDS, shuffle = False, random_state = SEED)
idxs = np.arange(0, len(DATA))

NET_PARAMS = {'n_classes': NCLASSES,
            'model': None,
            'domain': DOMAIN,
            'transform_method': METHOD,
            'scheduler_opt': True,
            'early_stopping': False,
            'validation_split': 0.0,
            'val_loss_min': None,
            'patience': None,
            'batch_size': 8,
            'epochs': 4,
            'dropout': 0.2,
            'MAX_SENT_LEN': 512,
            'lr': 2e-5,
            'RUNS': 10
            }

if GPU:
    MODELS =  ['bert']
else:
    MODELS =  [ 'svm', 'rf', 'nb-multinomial' ]
    CLF_PARAMS_DEFAULT = {}
    CLF_PARAMS_DEFAULT['svm'] = {'kernel':'linear', 'probability':True, 'C':SVC().get_params()['C'], 'random_state':SEED}
    CLF_PARAMS_DEFAULT['rf'] = {'n_estimators':10, 'criterion':'entropy', 'random_state':SEED}
    CLF_PARAMS_DEFAULT['nb-multinomial'] = MNB().get_params()

#out
if 'out' not in os.listdir( os.path.join( os.getcwd(), DOMAIN) ):
    os.mkdir( os.path.join( os.getcwd(), DOMAIN, 'out' ) )
#PL,AL.SSL
if 'BATCH'+str(BATCH) not in os.listdir( os.path.join( os.getcwd(), DOMAIN, 'out') ):
    os.mkdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH) ) )
if FILENAME not in os.listdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH)  ) ):
    os.mkdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH), FILENAME ) )
if 'PROB'+str(THR_PRB) not in os.listdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH)) ):
    os.mkdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH), 'PROB'+str(THR_PRB) ) )
if FILENAME not in os.listdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH), 'PROB'+str(THR_PRB) ) ):
    os.mkdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH), 'PROB'+str(THR_PRB), FILENAME ) )
#Results
if 'RESULTS' not in os.listdir( os.path.join( os.getcwd(), DOMAIN, 'out') ):
    os.mkdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'RESULTS' ) )
if FILENAME not in os.listdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'RESULTS' ) ):
    os.mkdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'RESULTS', FILENAME ) )
#Todo
for MODEL in MODELS:
    if MODEL not in os.listdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH), FILENAME ) ):
        os.mkdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH), FILENAME, MODEL ) )
    if MODEL not in os.listdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH), 'PROB'+str(THR_PRB), FILENAME ) ):
        os.mkdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH), 'PROB'+str(THR_PRB), FILENAME, MODEL ) )
    if MODEL not in os.listdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'RESULTS', FILENAME ) ):
        os.mkdir( os.path.join( os.getcwd(), DOMAIN, 'out', 'RESULTS', FILENAME, MODEL ) )

In [ ]:
if 'models' not in os.listdir( os.path.join( os.getcwd(), DOMAIN ) ) and GPU:
  os.mkdir( os.path.join( os.getcwd(), DOMAIN, 'models' ) )

if DOMAIN == 'texts' and GPU:
  if 'bert-base-multilingual-uncased' not in os.listdir( os.path.join( os.getcwd(), DOMAIN, 'models' ) ):
    !curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
    !sudo apt-get install git-lfs
    !git lfs install
    !git clone https://huggingface.co/bert-base-multilingual-uncased './texts/models/bert-base-multilingual-uncased'

In [ ]:
print('Filename:', FILENAME)
print('Domain:', DOMAIN)
print( 'Classes:', set( CLASSES ) )
print( 'Samples:', len(CLASSES), '\n' )
print( 'Batch:', BATCH, '\n' )
print('SSL-TYPE:', SSL_TYPE)
print('GRID-PARAMS:', GRID_PARAMS)
print('PERCENTAGE:', PGE)

for r in range(RUNS):
  idxs = shuffle(idxs, random_state = SEED)
  CLASSES = CLASSES[idxs]
  DATA = DATA[idxs]

  k = -1
  for train_index, test_index in folds.split(idxs):
    k+=1
    print( '\nrun:', r+1, 'fold: ', k+1 )
    print('Train:', len(train_index), 'Test:', len(test_index))
    print('AVG-train:', np.mean(CLASSES[train_index]), 'AVG-test:', np.mean(CLASSES[test_index]))

    X_train = copy.deepcopy( DATA[train_index] )
    y_train = copy.deepcopy( CLASSES[train_index] )
    X_test = copy.deepcopy( DATA[test_index] )
    y_test = copy.deepcopy( CLASSES[test_index] )

    for MODEL in MODELS:

      CLF_PARAMS = {}
      if 'bert' not in MODEL:
        if GRID_PARAMS and RESULTS_OPT:
          CLF_PARAMS = grid_search(X_train, y_train, FOLDS, MODEL, DOMAIN, METHOD)
        else:
            CLF_PARAMS = copy.deepcopy(CLF_PARAMS_DEFAULT[MODEL])
          
      results = defaultdict(list)
      results_lc_pl = defaultdict(list)
      results_lc_al = defaultdict(list)
      results_lc_sslal = defaultdict(list)

      NET_PARAMS['model'] = MODEL
      print('\n'+MODEL)

      if RESULTS_OPT:

        CURVE = 'RESULTS'
        print('-'*30+'\n'+CURVE)
        curve = Curves(X_train, y_train, X_test, y_test, BATCH, CURVE, MODEL, NET_PARAMS, CLF_PARAMS, GRID_PARAMS, FOLDS, SSL_TYPE, PGE)
        pred = curve.model_selection(MODEL, DATA[train_index], CLASSES[train_index], DATA[test_index])
        results[MODEL].append( [CLASSES[test_index], np.argmax(pred, axis=1)] )
        results[MODEL+'_params'].append(CLF_PARAMS)
        print('-'*30+'\nRESULTS')
        print( 'ACC (%):', 100*accuracy_score( CLASSES[test_index], np.argmax(pred, axis=1) ) )
        del curve
        gc.collect()
        torch.cuda.empty_cache() 
        # save model (fold)
        with open( os.path.join( os.getcwd(), DOMAIN, 'out', 'RESULTS', FILENAME, MODEL, FILENAME+'_results_'+MODEL+'_r'+str(r+1)+'_f'+str(k+1)+'_p'+str(GRID_PARAMS)+'.pkl' ), 'wb') as a:
          pickle.dump(results, a, protocol = 2)
        del pred
        del results
        gc.collect()
        torch.cuda.empty_cache() 

      if PL_AL_OPT:
          
        CURVE = 'PL'
        print('-'*30+'\n'+CURVE)
        curve = Curves(X_train, y_train, X_test, y_test, BATCH, CURVE, MODEL, NET_PARAMS, CLF_PARAMS, GRID_PARAMS, FOLDS, SSL_TYPE, PGE)
        curve.learningCurve()
        results_lc_pl[CURVE+'_batch'+str(BATCH)].append( [CLASSES[test_index], curve.results['y']  ] )
        results_lc_pl[MODEL+'_params'].append(curve.clf_params)
        del curve
        gc.collect()
        torch.cuda.empty_cache() 
        # save model (fold)
        with open( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH), FILENAME, MODEL, FILENAME+'_results_lc_pl_'+MODEL+'_r'+str(r+1)+'_f'+str(k+1)+'_p'+str(GRID_PARAMS)+'.pkl' ), 'wb') as a:
          pickle.dump(results_lc_pl, a, protocol = 2)
        del results_lc_pl
        gc.collect()
        torch.cuda.empty_cache() 
        
        CURVE = 'AL'
        print('-'*30+'\n'+CURVE)
        curve = Curves(X_train, y_train, X_test, y_test, BATCH, CURVE, MODEL, NET_PARAMS, CLF_PARAMS, GRID_PARAMS, FOLDS, SSL_TYPE, PGE)
        curve.learningCurve()
        results_lc_al[CURVE+'_batch'+str(BATCH)].append( [CLASSES[test_index], curve.results['y']  ] )
        results_lc_al['scores_'+CURVE+'_batch'+str(BATCH)].append( [curve.results['x'], curve.results['scores']  ] )
        results_lc_al[MODEL+'_params'].append(curve.clf_params)
        del curve
        gc.collect()
        torch.cuda.empty_cache() 
        # save model (fold) 
        with open( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH), FILENAME, MODEL, FILENAME+'_results_lc_al_'+MODEL+'_r'+str(r+1)+'_f'+str(k+1)+'_p'+str(GRID_PARAMS)+'.pkl' ), 'wb') as a:
          pickle.dump(results_lc_al, a, protocol = 2)
        del results_lc_al
        gc.collect()
        torch.cuda.empty_cache() 

      if SSL_OPT:

        CURVE = 'SSLAL'
        print('-'*30+'\n'+CURVE)
        curve = Curves(X_train, y_train, X_test, y_test, BATCH, CURVE, MODEL, NET_PARAMS, CLF_PARAMS, GRID_PARAMS, FOLDS, SSL_TYPE, PGE, THR_PRB)
        curve.learningCurve()
        results_lc_sslal[CURVE+'_batch'+str(BATCH)].append( [CLASSES[test_index], curve.results['y']  ] )
        results_lc_sslal['samples_'+CURVE+'_batch'+str(BATCH)].append( [ curve.results['x'], curve.results['samples'] ] )
        results_lc_sslal['distribution_'+CURVE+'_batch'+str(BATCH)].append( [ curve.results['x'], curve.results['distribution'] ] )
        results_lc_sslal['probs_qs_'+CURVE+'_batch'+str(BATCH)].append( [CLASSES[test_index], curve.results['probs_qs']  ] )
        results_lc_sslal['probs_val_'+CURVE+'_batch'+str(BATCH)].append( [CLASSES[test_index], curve.results['probs_val']  ] )
        results_lc_sslal['scores_'+CURVE+'_batch'+str(BATCH)].append( [curve.results['x'], curve.results['scores']  ] )
        results_lc_sslal[MODEL+'_params'].append(curve.clf_params)
        del curve
        gc.collect()
        torch.cuda.empty_cache() 
        # save model (fold)
        with open( os.path.join( os.getcwd(), DOMAIN, 'out', 'BATCH'+str(BATCH), 'PROB'+str(THR_PRB), FILENAME, MODEL, FILENAME+'_results_lc_sslal_'+MODEL+'_r'+str(r+1)+'_f'+str(k+1)+'_p'+str(GRID_PARAMS)+'_'+SSL_TYPE+'_'+str(PGE)+'.pkl' ), 'wb') as a:
          pickle.dump(results_lc_sslal, a, protocol = 2)
        del results_lc_sslal
        gc.collect()
        torch.cuda.empty_cache() 
      
    del X_train 
    del y_train 
    del X_test
    del y_test
    gc.collect()
    torch.cuda.empty_cache() 
